In [41]:
import pysam
import numpy as np
import cPickle as pickle
%load_ext autoreload
%autoreload 2
%matplotlib inline
from collections import Counter
import matplotlib.pyplot as plt
import logging
logging.basicConfig(level=logging.INFO)
from operator import itemgetter
import pandas as pd

In [42]:
import os

In [43]:
def fastq_to_names(in_file):
    with open(in_file) as fq:
        content = fq.readlines()
    
    names = []
    for line in content:
        if 'L183:338:CAGAAANXX' in line:
            names.append(line.rstrip()[1:])
    
    return names

In [29]:
names = fastq_to_names('/data/parastou/UMI/data/HG/RawReads/HGsample_AATGTA.fastq')

In [44]:
def umi_to_umis(in_file):
    with open(in_file) as fu:
        content = fu.readlines()
    
    umis = []
    for line in content:
        umis.append(line.rstrip())
    
    return umis

In [30]:
umis = umi_to_umis('/data/parastou/UMI/data/HG/RawReads/HGsample_AATGTA.umi')

In [45]:
def umi_to_name_map(fq_file, umi_file):
    
    names = fastq_to_names(fq_file)
    umis = umi_to_umis(umi_file)
    un_dict = dict(zip(names, umis))
    
    return un_dict

In [36]:
un = umi_to_name_map('/data/parastou/UMI/data/HG/RawReads/HGsample_AATGTA.fastq','/data/parastou/UMI/data/HG/RawReads/HGsample_AATGTA.umi')

In [49]:
def xm_tag_reads(bam_file, fq_file, umi_file):
    
    umi_name_map = umi_to_name_map(fq_file, umi_file)
    
    st = pysam.AlignmentFile(bam_file, "rb")
    
    out_file = "%s.tagged.bam" % os.path.splitext(bam_file)[0]
    tagged_reads = pysam.AlignmentFile('tmp.bam', 'wb', template=st)
    
    st.reset()
    reads = st.fetch(until_eof=True)
    for r in reads:
        if r.has_tag('RG'):
            r.set_tag('RG', None)
        xm = umi_name_map[r.query_name]
        r.tags += [('XM', xm)]
        tagged_reads.write(r)

    tagged_reads.close()
    pysam.sort("-o", out_file, 'tmp.bam')
    pysam.index(out_file)

In [51]:
ff = '/data/parastou/UMI/data/HG/RawReads/HGsample_AATGTA.fastq'
uf = '/data/parastou/UMI/data/HG/RawReads/HGsample_AATGTA.umi'
bf = '/data/parastou/Star-Lab/test/NStar10.Aligned.sorted.out.bam'

In [53]:
st = pysam.AlignmentFile('/data/parastou/Star-Lab/test/NStar10.Aligned.sorted.out.tagged.bam', 'rb')
reads = st.fetch(until_eof=True)
for r in reads:
    print r.query_name, r.get_tag('XM')

L183:338:CAGAAANXX:5:2103:19095:7288 TGGGGTTACT
L183:338:CAGAAANXX:4:2313:19904:5163 GTGCCAACCA
L183:338:CAGAAANXX:5:1214:15648:39769 GGGGGCCTAA
L183:338:CAGAAANXX:4:2312:20639:64473 ACCAATTCTT
L183:338:CAGAAANXX:5:1210:11806:84753 ATGGAATTGA
L183:338:CAGAAANXX:6:2306:18268:16213 GGGAATCACG
L183:338:CAGAAANXX:5:2103:19095:7288 TGGGGTTACT
L183:338:CAGAAANXX:4:2313:19904:5163 GTGCCAACCA
L183:338:CAGAAANXX:4:2312:20639:64473 ACCAATTCTT
L183:338:CAGAAANXX:5:1210:11806:84753 ATGGAATTGA
L183:338:CAGAAANXX:4:1209:6031:59693 GTTTCGCTGG
L183:338:CAGAAANXX:5:2201:13092:85186 GTTTCGCTGG
L183:338:CAGAAANXX:5:2214:4556:36649 GTTTCGCTGG
L183:338:CAGAAANXX:5:2302:13858:44581 GTTTCGCTGG
L183:338:CAGAAANXX:5:2315:10868:87585 GTTTCGCTGG
L183:338:CAGAAANXX:6:1208:13606:67980 GTTTCGCTGG
L183:338:CAGAAANXX:6:1209:17026:66927 GTTTCGCTGG
L183:338:CAGAAANXX:6:1214:2014:27629 GTTTCGCTGG
L183:338:CAGAAANXX:6:2106:2308:35155 GTTTCGCTGG
L183:338:CAGAAANXX:6:2209:13747:27405 GTTTCGCTGG
L183:338:CAGAAANXX:4:1112:21

KeyboardInterrupt: 